In [1]:
import os
os.chdir("DA-MRS/data/baby")

In [2]:
import json
import pandas as pd

df = pd.read_csv("baby.inter", sep = '\t')
df_del = pd.read_csv("baby_del.inter", sep = '\t')
df_del2 = pd.read_csv("baby_del_0.2.inter", sep = '\t')

In [3]:
import numpy as np
n_items = 7050
df1 = df[df['x_label'] == 0]
item_cnt_df = pd.DataFrame(data = {'itemID' : np.arange(n_items)})
item_cnt_df = item_cnt_df.join(df1[['userID', 'itemID']].groupby('itemID').count())
item_cnt_df.rename(columns = {'userID' : 'train_cnt'}, inplace = True)
item_cnt_df.fillna(0, inplace = True)
item_cnt_df.sort_values('train_cnt', inplace = True)

In [4]:
df2 = df[df['x_label'] == 2]
# item_cnt_df = pd.DataFrame(data = {'itemID' : np.arange(n_items)})
item_cnt_df = item_cnt_df.join(df2[['userID', 'itemID']].groupby('itemID').count())
item_cnt_df.rename(columns = {'userID' : 'test_cnt'}, inplace = True)
item_cnt_df.fillna(0, inplace = True)
item_cnt_df.sort_values('test_cnt', inplace = True)

In [5]:
items_group_cold = np.array(item_cnt_df[:int(n_items * 0.7)]['itemID'].tolist())
items_group_warm = np.array(item_cnt_df[int(n_items * 0.7):]['itemID'].tolist())

In [6]:
new_item_ratio = 0.2

np.random.seed(1111)
new_items_cold = np.random.choice(items_group_cold, size = int(new_item_ratio * len(items_group_cold)), replace = False)
new_items_warm = np.random.choice(items_group_warm, size = int(new_item_ratio * len(items_group_warm)), replace = False)
new_items = np.concatenate((new_items_cold, new_items_warm))
# np.save("new_items.npy", new_items)
# np.save("cold_items.npy", items_group_cold)
# np.save("warm_items.npy", items_group_warm)

In [7]:
all_items = np.arange(n_items)
old_items = np.setdiff1d(all_items, new_items)
old_items_cold = np.setdiff1d(items_group_cold, new_items_cold)
old_items_warm = np.setdiff1d(items_group_warm, new_items_warm)

In [17]:
missing_item_ratio = 0.8 #2/3

np.random.seed(1225) #2024
missing_items_nc = np.random.choice(new_items_cold, size = int(missing_item_ratio * len(new_items_cold)), replace = False)
missing_items_nw = np.random.choice(new_items_warm, size = int(missing_item_ratio * len(new_items_warm)), replace = False)
missing_items_oc = np.random.choice(old_items_cold, size = int(missing_item_ratio * len(old_items_cold)), replace = False)
missing_items_ow = np.random.choice(old_items_warm, size = int(missing_item_ratio * len(old_items_warm)), replace = False)
missing_items = np.concatenate((missing_items_nc, missing_items_nw, missing_items_oc, missing_items_ow))


In [18]:
def split_arr_(arr, k = 4) :
    res = []
    l = len(arr) / k
    check_ = 0
    for i in range(k) :
        from_, to_ = int(i * l), int((i+1) * l)
        res.append(arr[from_:to_])
        check_ += len(res[i])
    assert check_ == len(arr)
    print([len(i) for i in res])
    return res

mnc = split_arr_(missing_items_nc)
mnw = split_arr_(missing_items_nw)
moc = split_arr_(missing_items_oc)
mow = split_arr_(missing_items_ow)

missing_items_dict = {}
missing_items_dict['t'] = np.concatenate((mnc[0], mnw[0], moc[0], mow[0]))
missing_items_dict['v'] = np.concatenate((mnc[1], mnw[1], moc[1], mow[1]))
# missing_items_dict['all'] = np.concatenate((mnc[3], mnw[3], moc[3], mow[3]))
missing_items_dict['all'] = np.concatenate((mnc[2], mnw[2], moc[2], mow[2],
                                            mnc[3], mnw[3], moc[3], mow[3]))

[197, 197, 197, 198]
[84, 85, 84, 85]
[789, 790, 789, 790]
[338, 338, 338, 339]


In [19]:
np.save(f"missing_items_{missing_item_ratio}", missing_items_dict, allow_pickle = True)

In [19]:
complete_items = np.setdiff1d(all_items, missing_items)
old_items = np.setdiff1d(all_items, new_items)
print(f"Total : {len(all_items)}")
print(f"New Items : {len(new_items)} | {len(new_items) / len(all_items) : .3f}")
print(f"Cold Items : {len(items_group_cold)} | {len(items_group_cold) / len(all_items) : .3f}")
print(f"Complete Items : {len(complete_items)} | {len(complete_items) / len(all_items) :.3f}")
print(f"Missing  Items : {len(missing_items)} | {len(missing_items) / len(all_items) :.3f}")

Total : 7050
New Items : 1410 |  0.200
Cold Items : 4935 |  0.700
Complete Items : 4232 | 0.600
Missing  Items : 2818 | 0.400


In [13]:
# df[df.itemID.isin(new_items)].userID.nunique()

15445

In [16]:
# related_users = df[df.itemID.isin(new_items)].userID.unique()
print(df[((df.userID.isin(related_users)) & (df.x_label == 2))])
print(df[((df.userID.isin(related_users)) & (df.x_label == 2)) | (df.itemID.isin(new_items) & (df.x_label == 0))])

        userID  itemID  rating   timestamp  x_label
21           3    6320     5.0  1376697600        2
26           4       0     4.0  1396310400        2
42           6    5374     1.0  1374019200        2
59           8    3664     2.0  1401667200        2
73           9       0     5.0  1384819200        2
...        ...     ...     ...         ...      ...
160749   19438    6810     4.0  1388102400        2
160768   19440    6871     5.0  1393718400        2
160774   19441    6833     5.0  1388448000        2
160786   19443    6942     4.0  1404259200        2
160791   19444    6994     5.0  1393632000        2

[17659 rows x 5 columns]
        userID  itemID  rating   timestamp  x_label
19           3    2194     4.0  1376697600        0
21           3    6320     5.0  1376697600        2
24           4    5070     4.0  1396310400        0
26           4       0     4.0  1396310400        2
32           6     771     5.0  1374019200        0
...        ...     ...     ...        

In [14]:
print(f"New Items : {len(new_items)} | {len(new_items) / len(all_items) : .3f}")  
dummy_ = np.intersect1d(new_items, items_group_warm)
print(f"\tIn Warm Items : {len(dummy_)} | {len(dummy_) / len(new_items):.3f}")
dummy__ = np.intersect1d(dummy_, complete_items)
print(f"\t\tIn Complete Items : {len(dummy__)} | {len(dummy__) / len(dummy_):.3f}")
dummy__ = np.intersect1d(dummy_, missing_items)
print(f"\t\tIn Missing  Items : {len(dummy__)} | {len(dummy__) / len(dummy_):.3f}")

dummy___ = np.intersect1d(dummy__, missing_items_dict['t'])
print(f"\t\t\tIn Missing_t Items : {len(dummy___)} | {len(dummy___) / len(dummy__):.3f}")
dummy___ = np.intersect1d(dummy__, missing_items_dict['v'])
print(f"\t\t\tIn Missing_v Items : {len(dummy___)} | {len(dummy___) / len(dummy__):.3f}")

dummy___ = np.intersect1d(dummy__, missing_items_dict['all'])
print(f"\t\t\tIn Missing_tva Items : {len(dummy___)} | {len(dummy___) / len(dummy__):.3f}")

dummy_ = np.intersect1d(new_items, items_group_cold)
print(f"\tIn Cold Items : {len(dummy_)} | {len(dummy_) / len(new_items):.3f}")
dummy__ = np.intersect1d(dummy_, complete_items)
print(f"\t\tIn Complete Items : {len(dummy__)} | {len(dummy__) / len(dummy_):.3f}")
dummy__ = np.intersect1d(dummy_, missing_items)
print(f"\t\tIn Missing  Items : {len(dummy__)} | {len(dummy__) / len(dummy_):.3f}")

dummy___ = np.intersect1d(dummy__, missing_items_dict['t'])
print(f"\t\t\tIn Missing_t Items : {len(dummy___)} | {len(dummy___) / len(dummy__):.3f}")
dummy___ = np.intersect1d(dummy__, missing_items_dict['v'])
print(f"\t\t\tIn Missing_v Items : {len(dummy___)} | {len(dummy___) / len(dummy__):.3f}")

dummy___ = np.intersect1d(dummy__, missing_items_dict['all'])
print(f"\t\t\tIn Missing_tva Items : {len(dummy___)} | {len(dummy___) / len(dummy__):.3f}")

New Items : 1410 |  0.200
	In Warm Items : 423 | 0.300
		In Complete Items : 339 | 0.801
		In Missing  Items : 84 | 0.199
			In Missing_t Items : 21 | 0.250
			In Missing_v Items : 21 | 0.250
			In Missing_tva Items : 42 | 0.500
	In Cold Items : 987 | 0.700
		In Complete Items : 790 | 0.800
		In Missing  Items : 197 | 0.200
			In Missing_t Items : 49 | 0.249
			In Missing_v Items : 49 | 0.249
			In Missing_tva Items : 99 | 0.503


In [13]:
missing_items_dict =np.load("./missing_items_0.666.npy", allow_pickle = True).item()

In [14]:
missing_items_dict.keys()

dict_keys(['t', 'v', 'all'])

In [15]:
missing_items_1 = np.concatenate((missing_items_dict['t'], missing_items_dict['v']))
missing_items_2 = missing_items_dict['all']

In [17]:
new_items = np.load("./new_items.npy")

In [40]:
item_cnt_df['new_item'] = item_cnt_df['itemID'].isin(new_items).astype(int)
item_cnt_df['missing_modal'] = 0.0
item_cnt_df.loc[item_cnt_df['itemID'].isin(missing_items_1), 'missing_modal'] = 1 
item_cnt_df.loc[item_cnt_df['itemID'].isin(missing_items_2), 'missing_modal'] = 2 

In [42]:
item_cnt_df[['train_cnt', 'test_cnt', 'new_item']].groupby(['new_item']).agg(['sum', 'std', 'mean', 'max', 'min'])

train_cnt                                   test_cnt            \
               sum        std       mean    max  min      sum       std   
new_item                                                                  
0          95062.0  27.775196  16.854965  554.0  0.0  17517.0  5.380731   
1          23489.0  25.023182  16.658865  263.0  1.0   4165.0  4.401432   

                                
              mean    max  min  
new_item                        
0         3.105851  116.0  0.0  
1         2.953901   49.0  0.0

In [43]:
item_cnt_df[['train_cnt', 'test_cnt', 'missing_modal']].groupby(['missing_modal']).agg(['sum', 'std', 'mean', 'max', 'min'])

train_cnt                                   test_cnt            \
                    sum        std       mean    max  min      sum       std   
missing_modal                                                                  
0.0             41555.0  31.646547  17.682979  554.0  1.0   7680.0  6.111881   
1.0             39234.0  25.909245  16.695319  472.0  0.0   7105.0  4.866796   
2.0             37762.0  23.525670  16.068936  263.0  0.0   6897.0  4.475896   

                                     
                   mean    max  min  
missing_modal                        
0.0            3.268085  116.0  0.0  
1.0            3.023404   84.0  0.0  
2.0            2.934894   56.0  0.0